In [16]:
# define environment

# create random walker to sample episodes

# Monte Carlo

# TD(0)

import numpy as np
import matplotlib.pyplot as plt

In [17]:
# Define the MDP: [S, A, P, R, gamma]

def getBasicRandomWalkMDP():
    # set of states S:
    S = ['A', 'B', 'C', 'D', 'E']
    Sterminal = [0, 4]

    # set of actions A:
    A = ['left', 'right']

    # state transition probability tensor P^a_s,s'
    # P[action, from, to]
    P = np.zeros((len(A), len(S), len(S)))
    P[:,0,0] = 1
    P[:,4,4] = 1
    P[0,1,0] = 1
    P[0,2,1] = 1
    P[0,3,2] = 1
    P[1,1,2] = 1
    P[1,2,3] = 1
    P[1,3,4] = 1
#     print("state transitions for action 'left':")
#     print(P[0,:,:])
#     print("state transitions for action 'right':")
#     print(P[1,:,:])

    # reward function:
    R = np.zeros((len(S), len(A)))
    R[3,1] = 1
#     print("reward function R[state,action]:")
#     print(R)

    gamma = 0.9
    
    return {'S': S, 'Sterminal': Sterminal, 'A' : A, 'P' : P, 'R' : R, 'gamma' : gamma}

In [18]:
def getRandomWalkSample(mdp):
    currentState = np.random.randint(1, len(mdp['S']) - 1)
    sampledStates = [currentState]
    sampledRewards = []
    while True:
        sampledAction = np.random.randint(0,2)
        transitionProbs = mdp['P'][sampledAction, currentState, :]
        sampledState = np.where(np.random.multinomial(1, transitionProbs))[0][0]
        sampledReward = mdp['R'][currentState, sampledAction]
        sampledStates.append(sampledState)
        sampledRewards.append(sampledReward)
        currentState = sampledState
        if sampledState in mdp['Sterminal']:
            return sampledStates, sampledRewards

In [19]:
# every visit implementation of Monte Carlo estimator:
def getMonteCarloEstimator(mdp, numberOfEpisodes = 10000):
    stateCounter = np.zeros(len(mdp['S']))
    totalReturns = np.zeros(len(mdp['S']))
    for i in range(0,numberOfEpisodes):
        sampledStates, sampledRewards = getRandomWalkSample(mdp)
        
        
        
        # update counter and returns for all states of episode (every visit)    
        for i, state in enumerate(sampledStates):
            stateCounter[state] += 1
            
            # estimate total time-discounted return G_t:
            G_t = 0
            for t, reward in enumerate(sampledRewards[i:]):
                G_t += (mdp['gamma'] ** t) * reward
            totalReturns[state] += G_t
            
    return totalReturns / stateCounter

mdp = getBasicRandomWalkMDP()
print(getMonteCarloEstimator(mdp, 100))

[ 0.          0.18698562  0.37756373  0.66999836  0.        ]


In [20]:
# solve value function analytically:

# our policy is uniform random between 'left' and 'right':
P = 0.5 * (mdp['P'][0,:,:] + mdp['P'][1,:,:])
R = 0.5 * (mdp['R'][:,0] + mdp['R'][:,1])

v = np.linalg.inv(np.identity(len(mdp['S'])) - mdp['gamma'] * P).dot(R)

print(np.round(v, 5))

[-0.       0.17017  0.37815  0.67017  0.     ]


In [27]:
# TD(0) implementation:
def TDzero(mdp, numberOfEpisodes, alpha = 0.1):
    currentVs = np.zeros(len(mdp['S']))
    for i in range(0, numberOfEpisodes):
        sampledStates, sampledRewards = getRandomWalkSample(mdp)
        for j, reward in enumerate(sampledRewards):
            state = sampledStates[j]
            nextState = sampledStates[j+1]
            TDtarget = reward + mdp['gamma'] * currentVs[nextState]
            TDerror = TDtarget - currentVs[state]
            currentVs[state] = currentVs[state] + alpha * TDerror
    return currentVs

Vs = TDzero(mdp, 100, 0.02)
print(Vs)

[ 0.          0.05015666  0.19285067  0.49593255  0.        ]


In [40]:
#TD(lambda) implementation:
def TDlambda(mdp, numberOfEpisodes, alpha, lambd):
    currentVs = np.zeros(len(mdp['S']))
    for i in range(0, numberOfEpisodes):
        sampledStates, sampledRewards = getRandomWalkSample(mdp)
        eligibilityTraces = np.zeros(len(mdp['S']))
        for j, reward in enumerate(sampledRewards):
            state = sampledStates[j]
            nextState = sampledStates[j+1]
            eligibilityTraces = mdp['gamma'] * lambd * eligibilityTraces
            eligibilityTraces[state] += 1
            TDtarget = reward + mdp['gamma'] * currentVs[nextState]
            TDerror = TDtarget - currentVs[state]
            currentVs[state] = currentVs[state] + alpha * TDerror * eligibilityTraces[state]
    return currentVs

Vs = TDlambda(mdp, 100, 0.05, 0)
print(Vs)        

[ 0.          0.13328653  0.34370888  0.66850862  0.        ]


In [45]:
np.random.seed(42)
mdp = getBasicRandomWalkMDP()
print(getMonteCarloEstimator(mdp, 100))
print(TDzero(mdp, 100, 0.05))
print(TDlambda(mdp, 100, 0.05, 1))

[ 0.          0.13640726  0.27233484  0.61650113  0.        ]
[ 0.          0.13407358  0.29022057  0.65034728  0.        ]
[ 0.          0.19086406  0.43765975  0.78520283  0.        ]
